https://universaldependencies.org/

In [ ]:
!pip install DAWG stanza

# Разбор синтаксиса

In [1]:
import glob
import os
import stanza
from ipywidgets import IntProgress
from IPython.display import display, clear_output


FILES_COUNT = 1000000


if not 'nlp' in locals():
    stanza.download('ru')
    nlp = stanza.Pipeline('ru')
    clear_output(wait=True)


file_names = glob.iglob('/home/kukuruku/data/proza_ru/**/*.txt', recursive=True)


progress = IntProgress(min=0, max=FILES_COUNT) # instantiate the bar
display(progress)


i = 0


for file_name in file_names:
    stanza_file_name = f"{file_name}.stanza"
    
    if i > FILES_COUNT: break
    i += 1
        
    if os.path.isfile(stanza_file_name):
        continue
        
    progress.value = i
    progress.description = f"{i} of {FILES_COUNT}"
        

    with open(file_name, 'r') as file:
        with open(stanza_file_name, 'wb') as stanza_file:
            text = file.read()
            doc = nlp(text)
            stanza_file.write(doc.to_serialized())


progress.value = i
progress.description = "Done"

2022-02-27 14:06:13 INFO: Downloading default packages for language: ru (Russian)...
2022-02-27 14:06:14 INFO: File exists: /home/kukuruku/stanza_resources/ru/default.zip.
2022-02-27 14:06:24 INFO: Finished downloading models and saved to /home/kukuruku/stanza_resources.
2022-02-27 14:06:24 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |
| lemma     | syntagrus |
| depparse  | syntagrus |
| ner       | wikiner   |

2022-02-27 14:06:24 INFO: Use device: cpu
2022-02-27 14:06:24 INFO: Loading: tokenize
2022-02-27 14:06:24 INFO: Loading: pos
2022-02-27 14:06:24 INFO: Loading: lemma
2022-02-27 14:06:24 INFO: Loading: depparse
2022-02-27 14:06:24 INFO: Loading: ner
2022-02-27 14:06:26 INFO: Done loading processors!


IntProgress(value=0, max=1000000)

KeyboardInterrupt: 

# Чтерие готовых данных

In [ ]:
import glob
import os
import stanza
from ipywidgets import IntProgress
from IPython.display import display, clear_output

if not 'nlp' in locals():
    stanza.download('ru')
    nlp = stanza.Pipeline('ru')
    clear_output(wait=True)
    
    
file_names = glob.iglob('/home/kukuruku/data/proza_ru/**/*.txt', recursive=True)
    
    
for file_name in file_names:
    stanza_file_name = f"{file_name}.stanza"
    
    try:   
        with open(stanza_file_name, 'rb') as stanza_file:
            doc = stanza.Document.from_serialized(stanza_file.read())
            
            print(doc)
    except Exception:
        break
    

# Распознание сущностей

In [ ]:
import stanza
from IPython.display import clear_output


if not 'nlp' in locals():
    stanza.download('ru')
    nlp = stanza.Pipeline('ru')
    clear_output(wait=True)

    

def find_root_entity(words):
    root = None
    amods = []
    
    for w in words:
        if w.upos == "NOUN" and w.deprel == "root":
            root = w
            break
            
    if not root:
        return None
    
    for w in words:
        if w.deprel == "amod" and w.head == root.id:
            amods.append(w)

    return ' '.join([*[a.text for a in amods], root.text])


assert find_root_entity(nlp('белая курица').sentences[0].words) == 'белая курица'
assert find_root_entity(nlp('рябина красная').sentences[0].words) == 'красная рябина'

nlp('застежка липучка')
# assert find_root_entity(nlp('застежка липучка').sentences[0].words) == 'застежка липучка'


import dawg

words = ['hello', 'world', 'hello world', 'hello dawg']

base_dawg = dawg.DAWG(words)
completion_dawg = dawg.CompletionDAWG(words)